In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Feb  3 08:53:48 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P0    28W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


##  data set


## RNN4Rec LibRecommender

In [ ]:
!pip install --upgrade numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.24.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.7.3 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.24.1 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.1 which is incompatible.


In [ ]:
!pip uninstall pycocotools
!pip install  pycocotools

Found existing installation: pycocotools 2.0.6
Uninstalling pycocotools-2.0.6:
  Would remove:
    /usr/local/lib/python3.8/dist-packages/pycocotools-2.0.6.dist-info/*
    /usr/local/lib/python3.8/dist-packages/pycocotools/*
Proceed (Y/n)? n
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install LibRecommender==0.12.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 97.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 KB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for fst-pso: filename=fst_pso-1.8.1-py3-none-any.whl size=20443 sha256=ea9fe915060eaf498fb1128c0c6956f75280e550bcc9d0f1b745d694028f3756
  Stored in directory: /root/.cache/pip/wheels/6a/65/c4/d27eeee9ba3fc150a0dae150519591103b9e0dbffde3ae77dc
  Created wheel for miniful: filename=miniful-0.0.6-py3-none-any.whl size=3530 sha256=bf5dafa10e7fe2f5a28888fc3d70ca143bedc445b4c1a682d7eef459246a1f60
  Stored in directory: /root/.cache/pip/wheels/ba/d9/a0/ddd93af16d5855dd9bad417623e70948f

In [ ]:
from datetime import datetime
import os
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from scipy import sparse
from scipy.sparse import csc_matrix
from sklearn.decomposition import TruncatedSVD

np.random.seed(0)
import tensorflow as tf

from libreco.data import DatasetFeat, DataInfo, DatasetPure
from libreco.data import split_by_ratio_chrono,split_by_ratio
from libreco.algorithms import DeepFM
from libreco.evaluation import evaluate
from libreco.prediction import predict_data_with_feats

In [ ]:

col_user = ["user"]
col_item = ["item"]
col_dense = ["label"]

In [ ]:
!gdown https://drive.google.com/uc?id=1kl65YOvoSAMSgszQQbua2q4Zwe1HgPct
!unzip -o "data.zip"  -d  "/content"

movie_10k_df = pd.read_csv("movies_min.csv")
amazon_df =pd.read_csv('amazon_min.csv')
book_df=pd.read_csv("book_min.csv")
food_df = pd.read_csv('food_min.csv')
ecommerce_df=pd.read_csv('ecom_min.csv')
ciao_df=pd.read_csv('ciao_min.csv')
amazon_df=amazon_df[['userID',	'itemID',	'rating','time']]
amazon_df.columns=[ 'user', 'item', 'label','timestamp']
item_ids = list(set(list(amazon_df.item.unique())))
user_ids = list(set(list(amazon_df.user.unique())))
dict_users = {}
index = 0
for ids in sorted(user_ids):
    dict_users[ids] = index
    index += 1
dict_items = {}
index = 0
for ids in sorted(item_ids):
    dict_items[ids] = index
    index += 1
amazon_df['user'] = amazon_df.user.map(dict_users)
amazon_df['item'] = amazon_df.item.map(dict_items)

Downloading...
From: https://drive.google.com/uc?id=1kl65YOvoSAMSgszQQbua2q4Zwe1HgPct
To: /content/data.zip
100% 4.16M/4.16M [00:00<00:00, 327MB/s]
Archive:  data.zip
  inflating: /content/amazon_min.csv  
  inflating: /content/book_min.csv   
  inflating: /content/ciao_min.csv   
  inflating: /content/ecom_min.csv   
  inflating: /content/food_min.csv   
  inflating: /content/movies_min.csv  


### prepare dataset

In [ ]:
train, test = split_by_ratio(amazon_df, test_size=0.2)

In [ ]:
train_data, data_info =  DatasetPure.build_trainset(
        train,
       
        shuffle=False,
    )

In [ ]:
test_data =  DatasetPure.build_testset(test, shuffle=False)

In [ ]:
#eval_data =  DatasetPure.build_testset(test)
print(data_info)


n_users: 2952, n_items: 2661, data sparsity: 0.6742 %


In [ ]:
train_data.build_negative_samples(
        data_info, item_gen_mode="random", num_neg=1, seed=2020
    )
test_data.build_negative_samples(
        data_info, item_gen_mode="random", num_neg=1, seed=2222
    )


random neg item sampling elapsed: 0.044s
random neg item sampling elapsed: 0.008s


In [ ]:
import torch
from libreco.data import split_by_ratio_chrono, DatasetPure, DatasetFeat
from libreco.algorithms import (
   
    RNN4Rec,
    Caser,
    WaveNet,
    Item2Vec,
    DeepWalk,
)

    # # only do negative sampling on eval data
    # eval_data.build_negative_samples(
    #     data_info, item_gen_mode="random", num_neg=1, seed=2222
    # )
metrics = [  
        "precision",
        "recall",
        "map",
        "ndcg",
    ]


### RNN4Rec

In [ ]:
rnn = RNN4Rec(
        "ranking",
        data_info,
        rnn_type="gru",
        loss_type="cross_entropy",
        embed_size=32,
        n_epochs=20,
        lr=0.001,
        lr_decay=None,
        hidden_units="64,16",
        reg=None,
        batch_size=512,
        num_neg=1,
        dropout_rate=None,
        recent_num=20,
        tf_sess_config=None,
    )
rnn.fit(
        train_data,
        verbose=2,
        shuffle=True,
        eval_data=test_data,
        metrics=metrics,
    )

total params: 287,381 | embedding params: 258,453 | network params: 28,928
Training start time: 2023-02-03 08:18:56


train: 100%|██████████| 207/207 [00:13<00:00, 15.20it/s]


Epoch 1 elapsed: 13.644s
	 train_loss: 0.6665


eval_rec: 100%|██████████| 2855/2855 [00:00<00:00, 5975.47it/s]


	 eval precision@10: 0.0062
	 eval recall@10: 0.0190
	 eval map@10: 0.0119
	 eval ndcg@10: 0.0231


...

	 eval precision@10: 0.0139
	 eval recall@10: 0.0375
	 eval map@10: 0.0445
	 eval ndcg@10: 0.0645


train: 100%|██████████| 207/207 [00:12<00:00, 16.31it/s]


Epoch 20 elapsed: 12.713s
	 train_loss: 0.0135


eval_rec: 100%|██████████| 2855/2855 [00:00<00:00, 7768.14it/s]


	 eval precision@10: 0.0147
	 eval recall@10: 0.0418
	 eval map@10: 0.0454
	 eval ndcg@10: 0.0669


In [ ]:
print("recommendation: ", rnn.recommend_user(user=1, n_rec=7))

recommendation:  [(2290, 0.9999999), (2143, 0.99999976), (2229, 0.99999976), (2243, 0.99999976), (2521, 0.9999995), (2421, 0.9999994), (2473, 0.9999987)]


### Caser

In [ ]:
cas = Caser(
       "ranking",
        data_info,
        loss_type="cross_entropy",
        embed_size=32,
        n_epochs=20,
        lr=1e-4,
        lr_decay=None,
        reg=None,
        batch_size=512,
        num_neg=1,
        dropout_rate=0.0,
        use_bn=False,
        nh_filters=16,
        nv_filters=4,
        recent_num=10,
        tf_sess_config=None,
    )
cas.fit(
        train_data,
        verbose=2,
        shuffle=True,
        eval_data=test_data,
        metrics=metrics,
    )

total params: 390,225 | embedding params: 352,809 | network params: 37,416
Training start time: 2023-02-03 08:57:08


train: 100%|██████████| 207/207 [00:04<00:00, 47.20it/s] 


Epoch 1 elapsed: 4.415s
	 train_loss: 0.6928


eval_rec: 100%|██████████| 2855/2855 [00:00<00:00, 6124.59it/s]


	 eval precision@10: 0.0063
	 eval recall@10: 0.0187
	 eval map@10: 0.0332
	 eval ndcg@10: 0.0400


...

	 eval precision@10: 0.0114
	 eval recall@10: 0.0299
	 eval map@10: 0.0459
	 eval ndcg@10: 0.0600


train: 100%|██████████| 207/207 [00:01<00:00, 114.68it/s]


Epoch 20 elapsed: 1.824s
	 train_loss: 0.239


eval_rec: 100%|██████████| 2855/2855 [00:00<00:00, 7320.54it/s]


	 eval precision@10: 0.0116
	 eval recall@10: 0.0292
	 eval map@10: 0.0454
	 eval ndcg@10: 0.0597


In [ ]:
print("recommendation: ", cas.recommend_user(user=1, n_rec=7))

recommendation:  [(1986, 0.9871257), (2017, 0.98326725), (2088, 0.9675387), (2015, 0.9666409), (2229, 0.96572703), (2246, 0.95985097), (2478, 0.9583813)]
